In [ ]:
# %%capture
# %env DATABASE_USERNAME=kial
# %env DATABASE_PASSWORD=
# %env DATABASE_NAME=legal_db
# %env DATABASE_HOST=docker.for.mac.localhost
# %env DATABASE_PORT=5432
# %env SECRET_KEY=ops
# %env LEGAL_URL=https://legal-api-dev.pathfinder.gov.bc.ca/api/v1/businesses
# %env ACCOUNT_SVC_AUTH_URL=https://sso-dev.pathfinder.gov.bc.ca/auth/realms/fcf0kpqr/protocol/openid-connect/token
# %env ACCOUNT_SVC_CLIENT_ID=<account>
# %env ACCOUNT_SVC_CLIENT_SECRET=<secret>
# %env NAMEX_AUTH_SVC_URL=https://sso-dev.pathfinder.gov.bc.ca/auth/realms/sbc/protocol/openid-connect/token
# %env NAMEX_SERVICE_CLIENT_USERNAME=<account>
# %env NAMEX_SERVICE_CLIENT_SECRET=<secret>
# %env NAMEX_SVC_URL=https://namex-dev.pathfinder.gov.bc.ca/api/v1/
# %env AUTH_URL=https://auth-api-dev.pathfinder.gov.bc.ca/api/v1

In [1]:
%%capture
%env DATABASE_USERNAME=kial
%env DATABASE_PASSWORD=
%env DATABASE_NAME=legal_db
%env DATABASE_HOST=docker.for.mac.localhost
%env DATABASE_PORT=5432
%env SECRET_KEY=ops
%env LEGAL_URL=https://legal-api-dev.pathfinder.gov.bc.ca/api/v1/businesses
%env ACCOUNT_SVC_AUTH_URL=https://sso-dev.pathfinder.gov.bc.ca/auth/realms/fcf0kpqr/protocol/openid-connect/token
%env ACCOUNT_SVC_CLIENT_ID=<account>
%env ACCOUNT_SVC_CLIENT_SECRET=<secret>
%env NAMEX_AUTH_SVC_URL=https://sso-dev.pathfinder.gov.bc.ca/auth/realms/sbc/protocol/openid-connect/token
%env NAMEX_SERVICE_CLIENT_USERNAME=<account>
%env NAMEX_SERVICE_CLIENT_SECRET=<secret>
%env NAMEX_SVC_URL=https://namex-dev.pathfinder.gov.bc.ca/api/v1/
%env AUTH_URL=https://auth-api-dev.pathfinder.gov.bc.ca/api/v1

In [2]:
import datetime
import os
import flask
import sqlalchemy
import psycopg2
import simplejson
import pandas as pd
import matplotlib
import legal_api
import requests
from legal_api.models import Business
from legal_api.services.namex import NameXService
from legal_api.services.bootstrap import AccountService
from IPython.core.display import HTML
%load_ext sql
%config SqlMagic.displaylimit = 5
%config Application.log_level="ERROR"

Unable to configure logging, attempted conf:/usr/local/lib/python3.8/site-packages/legal_api/logging.conf


In [3]:
import flask
APP_CONFIG = legal_api.config.get_named_config(os.getenv('DEPLOYMENT_ENV', 'production'))
FLASK_APP = flask.Flask(__name__)
FLASK_APP.config.from_object(APP_CONFIG)
legal_api.db.init_app(FLASK_APP)
FLASK_APP.app_context().push()

In [ ]:
def stop_on_false(test: bool, test_name: str):
    # this will bail out of the execution if called by papermill
    #     failure_condition=True
    #     assert not test, test_name
    assert test, test_name

In [1]:
def stop_on_true(test: bool, test_name: str):
    # this will bail out of the execution if called by papermill
    #     failure_condition=True
    assert not test, test_name

In [4]:
from legal_api.models import Filing, db
from sqlalchemy import or_
def get_latest_correction_filing(business_id: str) -> Filing:
  filing_type='correction'
  expr = Filing._filing_json[('filing', filing_type)]
  max_filing = db.session.query(db.func.max(Filing._filing_date).label('last_filing_date')).\
            filter(Filing.business_id == business_id).\
            filter(or_(Filing._filing_type == filing_type,
                       expr.label('legal_filing_type').isnot(None))).\
            subquery()
  filings = Filing.query.join(max_filing, Filing._filing_date == max_filing.c.last_filing_date). \
            filter(Filing.business_id == business_id). \
            order_by(Filing.id.desc())

  return filings.first()

Unable to configure logging, attempted conf:/usr/local/lib/python3.8/site-packages/legal_api/logging.conf
